<a href="https://colab.research.google.com/github/SidouPrince/NeuralNetwork/blob/main/Anglais_Fran%C3%A7ais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LOAD & CLEANING DATA


In [3]:
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
import keras
from keras.preprocessing.text import Tokenizer
from collections import Counter
import numpy as np
import nltk
import tensorflow as tf
from nltk.corpus import stopwords
import pandas as pd
import pysrt
import re
#from helper import load_data

import os, sys

import matplotlib.pyplot as plt

In [ ]:
!pip install pysrt
!pip install helper

In [4]:
def load_english_sentences(file_name):
  english_sentences = []
  english_sentences_file = open(file_name)
  for sentence in english_sentences_file:
    english_sentences.append(sentence)
  return english_sentences

def load_french_sentences(file_name):
  french_sentences = []
  french_sentences_file = open(file_name)
  for sentence in french_sentences_file:
    french_sentences.append(sentence)
  return french_sentences

In [5]:
english_sentences = load_english_sentences('/content/small_vocab_en.txt')
french_sentences = load_french_sentences('/content/small_vocab_fr.txt')

In [6]:
print(english_sentences[1])
print("La traduction corespondante")
print(french_sentences[1])

the united states is usually chilly during july , and it is usually freezing in november .

La traduction corespondante
les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .



In [7]:
#La longueuur des deux fichier
print(len(english_sentences))
print(len(french_sentences))

137860
137860


In [8]:
english_words_counter = Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = Counter([word for sentence in french_sentences for word in sentence.split()])

In [9]:
print("English words {}".format(len([word for sentence in english_sentences for word in sentence.split()])))
print("English unique words {}".format(len(english_words_counter)))
print("10 most common words in english")
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print("French words {}".format(len([word for sentence in french_sentences for word in sentence.split()])))
print("French unique words {}".format(len(french_words_counter)))
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')


English words 1823250
English unique words 227
10 most common words in english
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

French words 1961295
French unique words 355
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [10]:
def tokenize(x):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(x)
  tok = tokenizer.texts_to_sequences(x)
  return tok, tokenizer

  
def pad(data, length=None):
  padding = tf.keras.preprocessing.sequence.pad_sequences(data, padding='post', maxlen=length)
  return padding

In [11]:
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']

tokenized, tokenizer = tokenize(text_sentences)
test_pad = pad(tokenized)

for sample_i, (token_sent, pad_sent) in enumerate(zip(tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [12]:
print(tokenizer.word_index)

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}


In [13]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import io
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader

In [14]:
def load_sentences(file_name):
  sentences = []
  sentences_file = open(file_name)
  for sentence in sentences_file:
    sentences.append(sentence)
  return sentences

In [68]:

pathAllVocab = ['/content/small_vocab_en.txt', '/content/small_vocab_fr.txt']
trainPaths = [ '/content/small_vocab_en_train.txt',   '/content/small_vocab_fr_train.txt']
testPaths = ['/content/small_vocab_en_test.txt', '/content/small_vocab_fr_test.txt']
validationPaths = ['/content/small_vocab_en_validate.txt',  '/content/small_vocab_fr_validate.txt']


_,fr_tokenizer = tokenize(load_sentences(pathAllVocab[1]))
_,en_tokenizer = tokenize(load_sentences(pathAllVocab[0]))
fr_vocab = fr_tokenizer.word_index
en_vocab = en_tokenizer.word_index


def data_process(filePath):

  frData = []
  enData = []
  myData = []

  #French sentences
  french_sentences = load_sentences(filePath[1])
  tokenized_fr_sentences,_ = tokenize(french_sentences)
  padded_fr = pad(tokenized_fr_sentences, 21)

  #English sentences
  english_sentences = load_sentences(filePath[0])
  tokenized_en_sentences,_ = tokenize(english_sentences)
  padded_en = pad(tokenized_en_sentences, 21)
  return padded_en, padded_fr

preproc_en_train, preproc_fr_train = data_process(trainPaths)
preproc_en_val, preproc_fr_val = data_process(validationPaths)
preproc_en_test, preproc_fr_test = data_process(testPaths)


In [89]:
french_vocab_size = len(fr_vocab)
english_vocab_size = len(en_vocab)

In [61]:
print(preproc_fr_train.shape)

(80000, 20)


In [18]:
tmp = pad(preproc_en_train, 20)

In [19]:
tmp.shape
tmp = tmp.reshape((-1, preproc_fr_train.shape[-2], 1))
print(tmp.shape)

(20, 80000, 1)


In [20]:
tmp_y = preproc_fr_train.reshape((preproc_fr_train.shape[-2],-1, 1))
print(tmp_y.shape)

(80000, 20, 1)


In [21]:
max_french_sequence_length = 20
max_french_sequence_length = 15

#Models

In [22]:
from keras.layers import Input
from keras.models import Sequential
from keras.layers import GRU, TimeDistributed, Dense,Embedding
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam

In [80]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    
   
    model = Sequential([GRU(100,input_shape=(21,1),return_sequences=True),TimeDistributed(Dense(345,activation='softmax'))])

    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(0.001))
    return model
tmp_x = preproc_en_train.reshape((preproc_fr_train.shape[-2],-1, 1))
tmp_y = preproc_fr_train.reshape((preproc_fr_train.shape[-2],-1, 1))



tmp_x_val = preproc_en_val.reshape((preproc_fr_val.shape[-2],-1, 1))
tmp_y_val = preproc_fr_val.reshape((preproc_fr_val.shape[-2],-1, 1))
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    199,
    344)
simple_rnn_model.summary()
history = simple_rnn_model.fit(tmp_x, tmp_y, epochs=10,batch_size=900,validation_split=0.2, validation_data=(tmp_x_val, tmp_y_val))

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 21, 100)           30900     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 21, 345)           34845     
Total params: 65,745
Trainable params: 65,745
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
72/72 [==============================] - 48s 634ms/step - loss: 4.5295 - val_loss: 2.5477
Epoch 2/10
72/72 [==============================] - 41s 573ms/step - loss: 2.4775 - val_loss: 2.2804
Epoch 3/10
72/72 [==============================] - 43s 592ms/step - loss: 2.2087 - val_loss: 1.9758
Epoch 4/10
72/72 [==============================] - 41s 568ms/step - loss: 1.9148 - val_loss: 1.7700
Epoch 5/10
72/72 [==============================] - 44s 610ms/step - loss: 1.7408 - val_loss: 1.6571
Epoch 

In [81]:
history.history

{'loss': [3.5306386947631836,
  2.416224718093872,
  2.1343350410461426,
  1.8652037382125854,
  1.714542031288147,
  1.6204372644424438,
  1.5490201711654663,
  1.489421010017395,
  1.4386987686157227,
  1.3952075242996216],
 'val_loss': [2.5477161407470703,
  2.2804393768310547,
  1.9758328199386597,
  1.7699960470199585,
  1.6570539474487305,
  1.576246738433838,
  1.5114405155181885,
  1.4564123153686523,
  1.409471035003662,
  1.3684992790222168]}

In [67]:
simple_rnn_model.evaluate()

(80000, 21, 1)
(80000, 20)


In [70]:
index_to_words = {id: word for word, id in fr_vocab.items()}
index_to_words[0] = '<pad>'
print(index_to_words)

{1: 'est', 2: 'en', 3: 'il', 4: 'les', 5: 'mais', 6: 'et', 7: 'la', 8: 'parfois', 9: 'jamais', 10: 'le', 11: "l'", 12: 'généralement', 13: 'moins', 14: 'aimé', 15: 'au', 16: 'fruit', 17: 'préféré', 18: 'agréable', 19: 'froid', 20: 'son', 21: 'chaud', 22: 'de', 23: 'plus', 24: 'automne', 25: 'mois', 26: 'à', 27: 'elle', 28: 'citrons', 29: 'paris', 30: 'inde', 31: 'unis', 32: 'états', 33: 'france', 34: 'jersey', 35: 'new', 36: 'chine', 37: 'pendant', 38: 'pamplemousse', 39: 'mon', 40: 'votre', 41: 'juin', 42: 'printemps', 43: 'janvier', 44: 'hiver', 45: 'mars', 46: 'été', 47: 'mai', 48: 'septembre', 49: 'juillet', 50: 'avril', 51: 'novembre', 52: 'décembre', 53: 'février', 54: 'octobre', 55: 'aime', 56: 'août', 57: 'merveilleux', 58: 'relaxant', 59: 'doux', 60: 'humide', 61: 'notre', 62: 'californie', 63: 'sec', 64: 'leur', 65: 'occupé', 66: 'pluvieux', 67: 'calme', 68: 'beau', 69: 'habituellement', 70: 'pommes', 71: 'pêches', 72: 'oranges', 73: 'poires', 74: 'fraises', 75: 'bananes', 76

In [74]:
tmp_x_test = preproc_en_test.reshape((preproc_en_test.shape[-2],-1, 1))
tmp_y_test = preproc_fr_test.reshape((preproc_fr_test.shape[-2],-1, 1))
prediction = simple_rnn_model.predict(tmp_x[:1])[0]
print(np.argmax(prediction, 1))
string = []
for idx in np.argmax(prediction, 1):
  string.append(index_to_words[idx])
print(string)

[ 7 33  1  8  8  2 25  5  3  1  1  2  2  0  0  0  0  0  0  0  0]
['la', 'france', 'est', 'parfois', 'parfois', 'en', 'mois', 'mais', 'il', 'est', 'est', 'en', 'en', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [87]:
result = simple_rnn_model.evaluate(tmp_x_test, tmp_y_test)

938/938 [==============================] - 11s 11ms/step - loss: 2.0367


In [88]:
print(result)
#print(tmp_y_test.shape)

2.0366780757904053
